In [ ]:
!nvidia-smi

Thu Feb 12 14:53:45 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install vllm

In [ ]:
from vllm import LLM, SamplingParams

# Use a small model that fits in T4's 15GB VRAM
llm = LLM(model="Qwen/Qwen2.5-1.5B-Instruct")

sampling_params = SamplingParams(
    temperature=0.7,
    top_p=0.9,
    max_tokens=256,
)

prompts = [
    "What is PagedAttention and why is it important?",
    "Explain transformer architecture in simple terms.",
    "Write a Python function to calculate fibonacci numbers.",
]

outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    prompt = output.prompt
    generated = output.outputs[0].text
    print(f"Prompt: {prompt[:50]}...")
    print(f"Output: {generated}")
    print("-" * 60)


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 02-12 14:54:46 [model.py:541] Resolved architecture: Qwen2ForCausalLM
WARNING 02-12 14:54:46 [model.py:1833] Your device 'Tesla T4' (with compute capability 7.5) doesn't support torch.bfloat16. Falling back to torch.float16 for compatibility.
WARNING 02-12 14:54:46 [model.py:1885] Casting torch.bfloat16 to torch.float16.
INFO 02-12 14:54:46 [model.py:1561] Using max model len 32768
INFO 02-12 14:54:47 [scheduler.py:226] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 02-12 14:54:47 [vllm.py:624] Asynchronous scheduling is enabled.
WARNING 02-12 14:54:47 [system_utils.py:140] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 02-12 14:55:44 [llm.py:343] Supported tasks: ['generate']


Adding requests:   0%|          | 0/3 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/3 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Prompt: What is PagedAttention and why is it important?...
Output:  PagedAttention is a neural network architecture used in natural language processing tasks, particularly in the field of language modeling. It is an efficient way to perform attention mechanisms, which are used to weigh and combine different parts of the input sequence to generate a final output.

The importance of PagedAttention lies in its ability to handle large input sequences more efficiently and effectively than traditional attention mechanisms. In traditional attention mechanisms, the attention weights are computed for each position in the input sequence independently. This can lead to a large number of attention computations, which can be computationally expensive and slow down the training process.

PagedAttention addresses this issue by dividing the input sequence into smaller chunks, called pages, and computing the attention weights for each page independently. This reduces the number of attention computation

In [ ]:
import os
os.environ["VLLM_LOGGING_LEVEL"] = "DEBUG"

import torch
from vllm import LLM, SamplingParams
import time

# Load model
llm = LLM(
    model="Qwen/Qwen2.5-1.5B-Instruct",
    gpu_memory_utilization=0.9,
    enable_prefix_caching=True,
    enforce_eager=True,
)

# GPU memory breakdown
total = torch.cuda.get_device_properties(0).total_memory / 1024**3
allocated = torch.cuda.memory_allocated(0) / 1024**3
reserved = torch.cuda.memory_reserved(0) / 1024**3

print("=" * 50)
print("GPU MEMORY BREAKDOWN")
print("=" * 50)
print(f"Total GPU Memory:     {total:.2f} GB")
print(f"PyTorch Allocated:    {allocated:.2f} GB")
print(f"PyTorch Reserved:     {reserved:.2f} GB")
print(f"Available for KV:     {total - reserved:.2f} GB")
print(f"Utilization:          {reserved/total*100:.1f}%")

# Prefix caching test
sp = SamplingParams(temperature=0.7, max_tokens=50)

print("\n" + "=" * 50)
print("PREFIX CACHING TEST")
print("=" * 50)

t0 = time.time()
out1 = llm.generate(["Explain how virtual memory works in OS."], sp)
t1 = time.time()
print(f"First  run: {t1-t0:.3f}s (cold - no cache)")

t0 = time.time()
out2 = llm.generate(["Explain how virtual memory works in OS."], sp)
t1 = time.time()
print(f"Second run: {t1-t0:.3f}s (warm - prefix cached!)")

t0 = time.time()
out3 = llm.generate(["What is the difference between TCP and UDP?"], sp)
t1 = time.time()
print(f"Third  run: {t1-t0:.3f}s (new prompt - no cache)")

INFO 02-12 15:01:12 [utils.py:261] non-default args: {'enable_prefix_caching': True, 'disable_log_stats': True, 'enforce_eager': True, 'model': 'Qwen/Qwen2.5-1.5B-Instruct'}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 02-12 15:01:13 [model.py:541] Resolved architecture: Qwen2ForCausalLM
WARNING 02-12 15:01:13 [model.py:1833] Your device 'Tesla T4' (with compute capability 7.5) doesn't support torch.bfloat16. Falling back to torch.float16 for compatibility.
WARNING 02-12 15:01:13 [model.py:1885] Casting torch.bfloat16 to torch.float16.
INFO 02-12 15:01:13 [model.py:1561] Using max model len 32768
INFO 02-12 15:01:14 [scheduler.py:226] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 02-12 15:01:14 [vllm.py:624] Asynchronous scheduling is enabled.
WARNING 02-12 15:01:14 [vllm.py:662] Enforce eager set, overriding optimization level to -O0
INFO 02-12 15:01:14 [vllm.py:762] Cudagraph is disabled under eager mode
WARNING 02-12 15:01:15 [system_utils.py:140] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

...

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Second run: 1.017s (warm - prefix cached!)


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Third  run: 1.045s (new prompt - no cache)


In [ ]:
import time
from vllm import LLM, SamplingParams

llm = LLM(
    model="Qwen/Qwen2.5-1.5B-Instruct",
    gpu_memory_utilization=0.8,
)

sp = SamplingParams(temperature=0.7, max_tokens=50)

long_prefix = """You are an expert computer scientist specializing in operating systems,
distributed systems, and computer architecture. You have deep knowledge of memory management,
virtual memory, paging, segmentation, TLB, cache hierarchies, and modern CPU architectures.
You have published multiple papers on these topics and have worked at major tech companies
implementing these systems at scale. When answering questions, provide detailed technical
explanations with concrete examples. Consider edge cases and performance implications.
Reference specific algorithms, data structures, and design patterns where relevant.
Here is a detailed technical document about memory management in modern operating systems
that you should use as context for your answers:
Memory management is one of the most critical functions of an operating system. The OS must
efficiently allocate and deallocate memory for processes, handle virtual-to-physical address
translation, manage page tables, handle page faults, implement page replacement algorithms
such as LRU, FIFO, Clock, and optimal replacement. The system must also handle shared memory,
copy-on-write semantics, memory-mapped files, demand paging, and working set management.
Modern systems use multi-level page tables (4-level in x86-64) to reduce memory overhead.
The Translation Lookaside Buffer (TLB) caches recent translations for performance. When a
TLB miss occurs, the hardware page table walker traverses the page table hierarchy. Huge
pages (2MB or 1GB) can reduce TLB pressure for large memory workloads. NUMA architectures
add complexity by having non-uniform memory access latencies across nodes.
Based on all of this context, answer the following question:
"""

prompt1 = long_prefix + "What is the role of TLB in virtual memory?"
prompt2 = long_prefix + "How does copy-on-write work?"
prompt3 = long_prefix + "Explain page replacement algorithms."

print("=" * 50)
print("PREFIX CACHING TEST (LONG PREFIX)")
print("=" * 50)

t0 = time.time()
llm.generate([prompt1], sp)
t1 = time.time()
print(f"Run 1 (cold):           {t1-t0:.3f}s")

t0 = time.time()
llm.generate([prompt2], sp)
t1 = time.time()
print(f"Run 2 (cached prefix):  {t1-t0:.3f}s")

t0 = time.time()
llm.generate([prompt3], sp)
t1 = time.time()
print(f"Run 3 (cached prefix):  {t1-t0:.3f}s")

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Run 2 (cached prefix):  0.797s


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Run 3 (cached prefix):  0.790s


In [ ]:
import time
from vllm import LLM, SamplingParams

llm = LLM(model="Qwen/Qwen2.5-1.5B-Instruct", gpu_memory_utilization=0.8)
sp = SamplingParams(temperature=0.7, max_tokens=100)

prompts = [
    "Explain how CPU caching works and why L1, L2, L3 matter.",
    "What is the difference between processes and threads?",
    "How does TCP ensure reliable data transmission?",
    "Explain the CAP theorem in distributed systems.",
    "What is a B-tree and why is it used in databases?",
    "How does garbage collection work in Java?",
    "Explain the difference between REST and GraphQL.",
    "What is consistent hashing and where is it used?",
    "How does TLS/SSL handshake work?",
    "Explain MapReduce with a simple example.",
    "What is a bloom filter and when would you use one?",
    "How does DNS resolution work step by step?",
    "Explain the difference between ACID and BASE.",
    "What is branch prediction in modern CPUs?",
    "How does a load balancer distribute traffic?",
    "Explain the Raft consensus algorithm.",
]

print(f"\n[TEST 1] Batch mode — all {len(prompts)} prompts at once")
t0 = time.time()
batch_outputs = llm.generate(prompts, sp)
t1 = time.time()
batch_time = t1 - t0
total_output_tokens = sum(len(o.outputs[0].token_ids) for o in batch_outputs)
print(f"  Time:       {batch_time:.2f}s")
print(f"  Throughput: {total_output_tokens/batch_time:.1f} tok/s")

print(f"\n[TEST 2] Sequential mode — one prompt at a time")
t0 = time.time()
seq_outputs = [llm.generate([p], sp)[0] for p in prompts]
t1 = time.time()
seq_time = t1 - t0
total_output_seq = sum(len(o.outputs[0].token_ids) for o in seq_outputs)
print(f"  Time:       {seq_time:.2f}s")
print(f"  Throughput: {total_output_seq/seq_time:.1f} tok/s")

print(f"\n{'='*50}")
print(f"  Speedup: {seq_time/batch_time:.2f}x faster")

INFO 02-12 15:20:30 [utils.py:261] non-default args: {'gpu_memory_utilization': 0.8, 'disable_log_stats': True, 'model': 'Qwen/Qwen2.5-1.5B-Instruct'}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 02-12 15:20:31 [model.py:541] Resolved architecture: Qwen2ForCausalLM
WARNING 02-12 15:20:31 [model.py:1833] Your device 'Tesla T4' (with compute capability 7.5) doesn't support torch.bfloat16. Falling back to torch.float16 for compatibility.
WARNING 02-12 15:20:31 [model.py:1885] Casting torch.bfloat16 to torch.float16.
INFO 02-12 15:20:31 [model.py:1561] Using max model len 32768
INFO 02-12 15:20:32 [scheduler.py:226] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 02-12 15:20:32 [vllm.py:624] Asynchronous scheduling is enabled.
WARNING 02-12 15:20:33 [system_utils.py:140] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 02-12 15:21:27 [llm.py:343] Supported tasks: ['generate']

[TEST 1] Batch mode — all 16 prompts at once


Adding requests:   0%|          | 0/16 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/16 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

...

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

  Time:       25.98s
  Throughput: 61.6 tok/s

  Speedup: 14.80x faster


In [ ]:
# Hugging Face
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"

prompts = [
    "Explain how CPU caching works and why L1, L2, L3 matter.",
    "What is the difference between processes and threads?",
    "How does TCP ensure reliable data transmission?",
    "Explain the CAP theorem in distributed systems.",
    "What is a B-tree and why is it used in databases?",
    "How does garbage collection work in Java?",
    "Explain the difference between REST and GraphQL.",
    "What is consistent hashing and where is it used?",
    "How does TLS/SSL handshake work?",
    "Explain MapReduce with a simple example.",
    "What is a bloom filter and when would you use one?",
    "How does DNS resolution work step by step?",
    "Explain the difference between ACID and BASE.",
    "What is branch prediction in modern CPUs?",
    "How does a load balancer distribute traffic?",
    "Explain the Raft consensus algorithm.",
]

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.padding_side = "left"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, dtype=torch.float16, device_map="cuda"
)
model.eval()

inputs = tokenizer(prompts, return_tensors="pt", padding=True).to("cuda")
torch.cuda.synchronize()

t0 = time.time()
with torch.no_grad():
    out = model.generate(
        **inputs, max_new_tokens=100,
        do_sample=True, temperature=0.7, top_p=0.9,
    )
torch.cuda.synchronize()
t1 = time.time()

total_out = sum(len(seq) - len(inputs["input_ids"][i]) for i, seq in enumerate(out))
mem = torch.cuda.max_memory_allocated() / 1024**3

print("=" * 50)
print("HuggingFace — 16 prompts batch (no PagedAttention)")
print("=" * 50)
print(f"Time:         {t1-t0:.2f}s")
print(f"Output toks:  {total_out}")
print(f"Throughput:   {total_out/(t1-t0):.1f} tok/s")
print(f"Peak GPU Mem: {mem:.2f} GB")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

HuggingFace — 16 prompts batch (no PagedAttention)
Time:         5.43s
Output toks:  1600
Throughput:   294.7 tok/s
Peak GPU Mem: 3.08 GB


In [ ]:
# paged attention
import time
import torch
from vllm import LLM, SamplingParams

MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"

prompts = [
    "Explain how CPU caching works and why L1, L2, L3 matter.",
    "What is the difference between processes and threads?",
    "How does TCP ensure reliable data transmission?",
    "Explain the CAP theorem in distributed systems.",
    "What is a B-tree and why is it used in databases?",
    "How does garbage collection work in Java?",
    "Explain the difference between REST and GraphQL.",
    "What is consistent hashing and where is it used?",
    "How does TLS/SSL handshake work?",
    "Explain MapReduce with a simple example.",
    "What is a bloom filter and when would you use one?",
    "How does DNS resolution work step by step?",
    "Explain the difference between ACID and BASE.",
    "What is branch prediction in modern CPUs?",
    "How does a load balancer distribute traffic?",
    "Explain the Raft consensus algorithm.",
]

llm = LLM(model=MODEL_NAME, gpu_memory_utilization=0.8)
sp = SamplingParams(temperature=0.7, top_p=0.9, max_tokens=100)

t0 = time.time()
out = llm.generate(prompts, sp)
t1 = time.time()

total_out = sum(len(o.outputs[0].token_ids) for o in out)
mem = torch.cuda.max_memory_allocated() / 1024**3

print("=" * 50)
print("vLLM — 16 prompts batch (with PagedAttention)")
print("=" * 50)
print(f"Time:         {t1-t0:.2f}s")
print(f"Output toks:  {total_out}")
print(f"Throughput:   {total_out/(t1-t0):.1f} tok/s")
print(f"Peak GPU Mem: {mem:.2f} GB")

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 02-12 15:24:39 [model.py:541] Resolved architecture: Qwen2ForCausalLM
WARNING 02-12 15:24:39 [model.py:1833] Your device 'Tesla T4' (with compute capability 7.5) doesn't support torch.bfloat16. Falling back to torch.float16 for compatibility.
WARNING 02-12 15:24:39 [model.py:1885] Casting torch.bfloat16 to torch.float16.
INFO 02-12 15:24:39 [model.py:1561] Using max model len 32768
INFO 02-12 15:24:40 [scheduler.py:226] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 02-12 15:24:40 [vllm.py:624] Asynchronous scheduling is enabled.
WARNING 02-12 15:24:41 [system_utils.py:140] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 02-12 15:25:34 [llm.py:343] Supported tasks: ['generate']


Adding requests:   0%|          | 0/16 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/16 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

vLLM — 16 prompts batch (with PagedAttention)
Time:         2.10s
Output toks:  1600
Throughput:   762.7 tok/s
Peak GPU Mem: 0.00 GB
